In [1]:
import yfinance as yf

sp500 = yf.download('^GSPC', start='2018-01-01', end='2022-12-31')
sp500.head()

sp500.to_csv('sp500_2018_2022.csv')

C:\Users\flare\AppData\Local\Temp\ipykernel_1416\1736441320.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp500 = yf.download('^GSPC', start='2018-01-01', end='2022-12-31')
[*********************100%***********************]  1 of 1 completed


PermissionError: [Errno 13] Permission denied: 'sp500_2018_2022.csv'

In [4]:
import pandas as pd
import sqlite3

# -----------------------------------------------------
# ① MultiIndex CSV を読み込む（あなたのCSV構造に対応）
# -----------------------------------------------------

df = pd.read_csv("sp500_2018_2022.csv", header=[0,1])  # ← 2段ヘッダを読み込む

# df.columns の例：
# MultiIndex([('Price','Close'), ('Price','High'), ...])

# -----------------------------------------------------
# ② 列名をフラット化（Close, High, Low, Open, Volume）
# -----------------------------------------------------

df.columns = df.columns.get_level_values(1)   # 2段目を採用（^GSPC）
df.columns = ["Date", "Close", "High", "Low", "Open", "Volume"]

# -----------------------------------------------------
# ③ Date を文字列に統一（念のため）
# -----------------------------------------------------
df["Date"] = df["Date"].astype(str)

# -----------------------------------------------------
# ④ SQLite に保存
# -----------------------------------------------------

conn = sqlite3.connect("sp500.db")
df.to_sql("sp500", conn, if_exists="replace", index=False)
conn.close()

print("SQLite sp500.db に登録完了")


SQLite sp500.db に登録完了


PS D:\cit\db2025> .\sqlite3.exe .\sp500.db
SQLite version 3.39.3 2022-09-05 11:02:23
Enter ".help" for usage hints.
sqlite> .tables
sp500
sqlite> SELECT Date, High-Low AS DailyRange FROM sp500 ORDER BY Date LIMIT 5;
2018-01-02|13.52978515625
2018-01-03|16.60009765625
2018-01-04|10.219970703125
2018-01-05|15.530029296875
2018-01-08|10.909912109375
sqlite> SELECT substr(Date,1,7) AS Month, AVG(Close) AS AvgClose FROM sp500 GROUP BY Month ORDER BY Month LIMIT 5;
2018-01|2789.8038155692
2018-02|2705.15524773849
2018-03|2702.77382114955
2018-04|2653.62520926339
2018-05|2701.49361905185